# FastHTML Components

> Interactive UI components for RBE demonstrations using FastHTML and MonsterUI

In [ ]:
#| default_exp rbe.components

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export
from fasthtml.common import *
from monsterui.all import *
import numpy as np
from typing import Dict, List, Optional
from technical_blog.rbe.probability import normalize, entropy
from technical_blog.rbe.bayes_core import update

## Bayes Calculator Component

In [ ]:
#| export
def bayes_calculator():
    "Interactive Bayes theorem calculator"
    return Card(
        H3("Bayes' Theorem Calculator", cls="text-xl font-bold mb-4"),
        
        # Scenario selector
        Div(
            Label("Scenario:", cls="block text-sm font-medium mb-2"),
            Select(
                Option("Network Intrusion", value="intrusion"),
                Option("Malware Detection", value="malware"),
                Option("Insider Threat", value="insider"),
                Option("Custom", value="custom"),
                id="scenario-select",
                cls="w-full p-2 border rounded"
            ),
            cls="mb-4"
        ),
        
        # Prior probability
        Div(
            Label("Prior P(Attack):", cls="block text-sm font-medium mb-2"),
            Div(
                Input(
                    type="range", 
                    id="prior-slider", 
                    min="0.01", 
                    max="0.99", 
                    step="0.01", 
                    value="0.05",
                    cls="w-full"
                ),
                Span(id="prior-display", cls="text-sm font-mono ml-2"),
                cls="flex items-center"
            ),
            cls="mb-4"
        ),
        
        # Likelihood inputs
        Div(
            Label("P(Evidence|Attack):", cls="block text-sm font-medium mb-2"),
            Div(
                Input(
                    type="range", 
                    id="like-attack-slider", 
                    min="0.01", 
                    max="0.99", 
                    step="0.01", 
                    value="0.90",
                    cls="w-full"
                ),
                Span(id="like-attack-display", cls="text-sm font-mono ml-2"),
                cls="flex items-center"
            ),
            cls="mb-4"
        ),
        
        Div(
            Label("P(Evidence|Normal):", cls="block text-sm font-medium mb-2"),
            Div(
                Input(
                    type="range", 
                    id="like-normal-slider", 
                    min="0.01", 
                    max="0.99", 
                    step="0.01", 
                    value="0.10",
                    cls="w-full"
                ),
                Span(id="like-normal-display", cls="text-sm font-mono ml-2"),
                cls="flex items-center"
            ),
            cls="mb-6"
        ),
        
        # Results
        Div(
            H4("Results:", cls="text-lg font-semibold mb-3"),
            Div(id="bayes-results", cls="space-y-2"),
            cls="p-4 bg-gray-50 rounded"
        ),
        
        # JavaScript
        Script(bayes_calculator_js()),
        
        cls="p-6"
    )

In [ ]:
#| export
def bayes_calculator_js():
    "JavaScript for Bayes calculator"
    return """
    function updateBayesCalc() {
        const prior = parseFloat(document.getElementById('prior-slider').value);
        const likeAttack = parseFloat(document.getElementById('like-attack-slider').value);
        const likeNormal = parseFloat(document.getElementById('like-normal-slider').value);
        
        // Update displays
        document.getElementById('prior-display').textContent = (prior * 100).toFixed(1) + '%';
        document.getElementById('like-attack-display').textContent = (likeAttack * 100).toFixed(1) + '%';
        document.getElementById('like-normal-display').textContent = (likeNormal * 100).toFixed(1) + '%';
        
        // Calculate Bayes
        const evidence = prior * likeAttack + (1 - prior) * likeNormal;
        const posterior = (prior * likeAttack) / evidence;
        
        // Calculate metrics
        const priorEntropy = -prior * Math.log2(prior) - (1-prior) * Math.log2(1-prior);
        const postEntropy = -posterior * Math.log2(posterior) - (1-posterior) * Math.log2(1-posterior);
        const infoGain = priorEntropy - postEntropy;
        const bayesFactor = likeAttack / likeNormal;
        
        // Update results
        document.getElementById('bayes-results').innerHTML = `
            <div class="grid grid-cols-2 gap-4">
                <div>
                    <p class="text-sm text-gray-600">Prior:</p>
                    <p class="text-lg font-semibold">${(prior * 100).toFixed(1)}%</p>
                </div>
                <div>
                    <p class="text-sm text-gray-600">Posterior:</p>
                    <p class="text-lg font-semibold text-red-600">${(posterior * 100).toFixed(1)}%</p>
                </div>
                <div>
                    <p class="text-sm text-gray-600">Bayes Factor:</p>
                    <p class="text-lg font-semibold">${bayesFactor.toFixed(2)}</p>
                </div>
                <div>
                    <p class="text-sm text-gray-600">Info Gain:</p>
                    <p class="text-lg font-semibold">${infoGain.toFixed(3)} bits</p>
                </div>
            </div>
        `;
    }
    
    // Scenario presets
    document.getElementById('scenario-select').addEventListener('change', function() {
        const presets = {
            'intrusion': {prior: 0.05, likeAttack: 0.90, likeNormal: 0.10},
            'malware': {prior: 0.15, likeAttack: 0.85, likeNormal: 0.05},
            'insider': {prior: 0.02, likeAttack: 0.70, likeNormal: 0.20},
            'custom': {prior: 0.50, likeAttack: 0.80, likeNormal: 0.20}
        };
        
        const preset = presets[this.value];
        if (preset) {
            document.getElementById('prior-slider').value = preset.prior;
            document.getElementById('like-attack-slider').value = preset.likeAttack;
            document.getElementById('like-normal-slider').value = preset.likeNormal;
            updateBayesCalc();
        }
    });
    
    // Event listeners
    document.getElementById('prior-slider').addEventListener('input', updateBayesCalc);
    document.getElementById('like-attack-slider').addEventListener('input', updateBayesCalc);
    document.getElementById('like-normal-slider').addEventListener('input', updateBayesCalc);
    
    // Initial calculation
    updateBayesCalc();
    """

## Particle Filter Visualization

In [ ]:
#| export
def particle_viz():
    "Interactive particle filter visualization"
    return Card(
        H3("Particle Filter Demo", cls="text-xl font-bold mb-4"),
        
        # Controls
        Div(
            Button("Start", id="pf-start", cls="btn btn-primary mr-2"),
            Button("Step", id="pf-step", cls="btn btn-secondary mr-2"),
            Button("Reset", id="pf-reset", cls="btn btn-outline"),
            cls="mb-4"
        ),
        
        # Canvas for visualization
        Canvas(id="pf-canvas", width="600", height="400", 
               cls="border rounded w-full"),
        
        # Info display
        Div(
            Div(id="pf-info", cls="text-sm"),
            cls="mt-4 p-3 bg-gray-50 rounded"
        ),
        
        # JavaScript
        Script(particle_viz_js()),
        
        cls="p-6"
    )

In [ ]:
#| export
def particle_viz_js():
    "JavaScript for particle filter visualization"
    return """
    class ParticleFilter {
        constructor(canvas) {
            this.canvas = canvas;
            this.ctx = canvas.getContext('2d');
            this.particles = [];
            this.weights = [];
            this.observation = {x: 300, y: 200};
            this.time = 0;
            this.nParticles = 100;
            
            this.init();
        }
        
        init() {
            // Initialize particles randomly
            this.particles = [];
            this.weights = [];
            
            for (let i = 0; i < this.nParticles; i++) {
                this.particles.push({
                    x: Math.random() * this.canvas.width,
                    y: Math.random() * this.canvas.height,
                    vx: (Math.random() - 0.5) * 2,
                    vy: (Math.random() - 0.5) * 2
                });
                this.weights.push(1 / this.nParticles);
            }
        }
        
        step() {
            // Predict: move particles
            for (let p of this.particles) {
                p.x += p.vx + (Math.random() - 0.5) * 5;
                p.y += p.vy + (Math.random() - 0.5) * 5;
                
                // Bounce off walls
                if (p.x < 0 || p.x > this.canvas.width) p.vx *= -1;
                if (p.y < 0 || p.y > this.canvas.height) p.vy *= -1;
            }
            
            // Update: weight by distance to observation
            let sumWeights = 0;
            for (let i = 0; i < this.particles.length; i++) {
                const p = this.particles[i];
                const dist = Math.sqrt(
                    Math.pow(p.x - this.observation.x, 2) + 
                    Math.pow(p.y - this.observation.y, 2)
                );
                this.weights[i] = Math.exp(-dist / 50);
                sumWeights += this.weights[i];
            }
            
            // Normalize weights
            for (let i = 0; i < this.weights.length; i++) {
                this.weights[i] /= sumWeights;
            }
            
            // Resample if needed
            const effSize = 1 / this.weights.reduce((sum, w) => sum + w*w, 0);
            if (effSize < this.nParticles / 2) {
                this.resample();
            }
            
            this.time++;
            this.draw();
            this.updateInfo();
        }
        
        resample() {
            const newParticles = [];
            const cumSum = [];
            let sum = 0;
            
            for (let w of this.weights) {
                sum += w;
                cumSum.push(sum);
            }
            
            for (let i = 0; i < this.nParticles; i++) {
                const r = Math.random();
                const idx = cumSum.findIndex(cs => cs >= r);
                newParticles.push({...this.particles[idx]});
            }
            
            this.particles = newParticles;
            this.weights = new Array(this.nParticles).fill(1 / this.nParticles);
        }
        
        draw() {
            this.ctx.clearRect(0, 0, this.canvas.width, this.canvas.height);
            
            // Draw observation
            this.ctx.fillStyle = 'red';
            this.ctx.beginPath();
            this.ctx.arc(this.observation.x, this.observation.y, 10, 0, 2 * Math.PI);
            this.ctx.fill();
            
            // Draw particles
            for (let i = 0; i < this.particles.length; i++) {
                const p = this.particles[i];
                const w = this.weights[i];
                
                this.ctx.fillStyle = `rgba(0, 0, 255, ${Math.min(w * 20, 1)})`;
                this.ctx.beginPath();
                this.ctx.arc(p.x, p.y, 3 + w * 100, 0, 2 * Math.PI);
                this.ctx.fill();
            }
        }
        
        updateInfo() {
            const effSize = 1 / this.weights.reduce((sum, w) => sum + w*w, 0);
            const maxWeight = Math.max(...this.weights);
            
            document.getElementById('pf-info').innerHTML = `
                <strong>Time:</strong> ${this.time} | 
                <strong>Effective Size:</strong> ${effSize.toFixed(1)} / ${this.nParticles} | 
                <strong>Max Weight:</strong> ${maxWeight.toFixed(4)}
            `;
        }
    }
    
    // Initialize
    const canvas = document.getElementById('pf-canvas');
    const pf = new ParticleFilter(canvas);
    let running = false;
    let interval = null;
    
    document.getElementById('pf-start').addEventListener('click', () => {
        if (!running) {
            running = true;
            interval = setInterval(() => pf.step(), 100);
            document.getElementById('pf-start').textContent = 'Stop';
        } else {
            running = false;
            clearInterval(interval);
            document.getElementById('pf-start').textContent = 'Start';
        }
    });
    
    document.getElementById('pf-step').addEventListener('click', () => {
        if (!running) pf.step();
    });
    
    document.getElementById('pf-reset').addEventListener('click', () => {
        running = false;
        clearInterval(interval);
        document.getElementById('pf-start').textContent = 'Start';
        pf.init();
        pf.draw();
        pf.updateInfo();
    });
    
    // Initial draw
    pf.draw();
    pf.updateInfo();
    
    // Move observation with mouse
    canvas.addEventListener('mousemove', (e) => {
        const rect = canvas.getBoundingClientRect();
        pf.observation.x = e.clientX - rect.left;
        pf.observation.y = e.clientY - rect.top;
        if (!running) pf.draw();
    });
    """

## Recursive Update Visualizer

In [ ]:
#| export
def recursive_update_viz():
    "Interactive recursive Bayesian updating visualization"
    return Card(
        H3("Recursive Bayesian Updating", cls="text-xl font-bold mb-4"),
        
        # Evidence sequence
        Div(
            H4("Evidence Sequence:", cls="text-lg font-semibold mb-2"),
            Div(id="evidence-sequence", cls="flex gap-2 flex-wrap mb-4"),
            cls="mb-4"
        ),
        
        # Current belief state
        Div(
            H4("Current Belief:", cls="text-lg font-semibold mb-2"),
            Div(id="belief-display", cls="h-32"),
            cls="mb-4 p-4 bg-gray-50 rounded"
        ),
        
        # Controls
        Div(
            Button("Add Port Scan", 
                   hx_post="/add_evidence",
                   hx_vals='{"type": "port_scan"}',
                   hx_target="#update-container",
                   cls="btn btn-sm btn-warning mr-2"),
            Button("Add Failed Login", 
                   hx_post="/add_evidence",
                   hx_vals='{"type": "failed_login"}',
                   hx_target="#update-container",
                   cls="btn btn-sm btn-warning mr-2"),
            Button("Add Normal Traffic", 
                   hx_post="/add_evidence",
                   hx_vals='{"type": "normal_traffic"}',
                   hx_target="#update-container",
                   cls="btn btn-sm btn-success mr-2"),
            Button("Reset", 
                   hx_post="/reset_belief",
                   hx_target="#update-container",
                   cls="btn btn-sm btn-outline"),
            cls="mb-4"
        ),
        
        # Update container
        Div(id="update-container"),
        
        cls="p-6"
    )

## Threat Dashboard

In [ ]:
#| export
def threat_dashboard():
    "Comprehensive threat monitoring dashboard"
    return Div(
        H2("Threat Monitoring Dashboard", cls="text-2xl font-bold mb-6"),
        
        # Metrics row
        Div(
            metric_card("Active Threats", "3", "text-red-600", "↑ 2"),
            metric_card("Alerts Today", "47", "text-yellow-600", "↓ 5"),
            metric_card("System Health", "94%", "text-green-600", "→ 0"),
            metric_card("Avg Response", "2.3s", "text-blue-600", "↓ 0.1s"),
            cls="grid grid-cols-1 md:grid-cols-2 lg:grid-cols-4 gap-4 mb-6"
        ),
        
        # Main content
        Div(
            # Left: Real-time monitor
            Div(
                Card(
                    H3("Real-time Activity", cls="text-lg font-bold mb-4"),
                    Canvas(id="activity-canvas", width="100%", height="300",
                          cls="w-full"),
                    cls="p-4 h-full"
                ),
                cls="lg:col-span-2"
            ),
            
            # Right: Alert feed
            Div(
                Card(
                    H3("Recent Alerts", cls="text-lg font-bold mb-4"),
                    Div(
                        id="alert-feed",
                        cls="space-y-2 h-64 overflow-y-auto"
                    ),
                    cls="p-4 h-full"
                ),
                cls="lg:col-span-1"
            ),
            
            cls="grid grid-cols-1 lg:grid-cols-3 gap-4"
        ),
        
        # Auto-refresh
        Div(hx_get="/refresh_dashboard", 
            hx_trigger="every 5s",
            hx_target="#alert-feed")
    )

def metric_card(title, value, color_class, change):
    "Individual metric card for dashboard"
    return Card(
        Div(
            P(title, cls="text-sm text-gray-600"),
            P(value, cls=f"text-3xl font-bold {color_class}"),
            P(change, cls="text-sm text-gray-500"),
            cls="text-center"
        ),
        cls="p-4"
    )

## Export

In [ ]:
#| export
__all__ = [
    # Calculator components
    'bayes_calculator', 'bayes_calculator_js',
    
    # Visualizations
    'particle_viz', 'particle_viz_js',
    'recursive_update_viz',
    
    # Dashboard
    'threat_dashboard', 'metric_card'
]